In [1]:
!pip install opencv-python numpy matplotlib

In [11]:
# Import necessary libraries
import numpy as np
import cv2
from collections import deque

# Define color range for object detection (e.g., blue color in HSV)
blueLower = np.array([100, 60, 60])
blueUpper = np.array([140, 255, 255])

# Define kernel for morphological operations (used for noise removal)
kernel = np.ones((5, 5), np.uint8)

# Define deques to store points for drawing with different colors
bpoints = [deque(maxlen=512)]  # Blue points
gpoints = [deque(maxlen=512)]  # Green points
rpoints = [deque(maxlen=512)]  # Red points
ypoints = [deque(maxlen=512)]  # Yellow points

# Initialize indices to track points in each color deque
bindex = 0
gindex = 0
rindex = 0
yindex = 0

# Define color values in BGR format (Blue, Green, Red, Yellow)
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255)]

# Initialize the current color index
colorIndex = 0



In [16]:
import cv2
import numpy as np

# Initialize the webcam
camera = cv2.VideoCapture(0)

# Loop over the frames from the video stream
while True:
    # Grab the current frame
    grabbed, frame = camera.read()

    # If the frame was not grabbed (end of the video stream), break from the loop
    if not grabbed:
        break

    # Flip the frame horizontally to avoid mirror effect
    frame = cv2.flip(frame, 1)

    # Convert the frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Add a paint interface to the camera feed (rectangles with color choices)
    # Create buttons for color choices and the clear button
    frame = cv2.rectangle(frame, (40, 1), (140, 65), (122, 122, 122), -1)  # Clear button
    frame = cv2.rectangle(frame, (160, 1), (255, 65), (255, 0, 0), -1)     # Blue color button
    frame = cv2.rectangle(frame, (275, 1), (370, 65), (0, 255, 0), -1)     # Green color button
    frame = cv2.rectangle(frame, (390, 1), (485, 65), (0, 0, 255), -1)     # Red color button
    frame = cv2.rectangle(frame, (505, 1), (600, 65), (0, 255, 255), -1)   # Yellow color button

    # Add text labels on top of the rectangles
    cv2.putText(frame, "CLEAR ALL", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                (150, 150, 150), 2, cv2.LINE_AA)

    # Display the resulting frame with the paint interface
    cv2.imshow("Paint Application", frame)

    # Check for keypresses to exit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the camera and close all OpenCV windows
camera.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [19]:
import cv2
import numpy as np

# Initialize the webcam
camera = cv2.VideoCapture(0)

# Define the color range for blue in HSV space
blueLower = np.array([100, 60, 60])
blueUpper = np.array([140, 255, 255])

# Define the kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Loop through frames from the webcam
while True:
    # Capture the current frame
    grabbed, frame = camera.read()

    # Break the loop if the frame is not grabbed (e.g., the camera is disconnected)
    if not grabbed:
        break

    # Flip the frame horizontally to avoid mirror effect
    frame = cv2.flip(frame, 1)

    # Convert the frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create a binary mask where the blue colors are white and everything else is black
    blueMask = cv2.inRange(hsv, blueLower, blueUpper)

    # Perform morphological operations to reduce noise in the binary mask
    blueMask = cv2.erode(blueMask, kernel, iterations=2)
    blueMask = cv2.morphologyEx(blueMask, cv2.MORPH_OPEN, kernel)
    blueMask = cv2.dilate(blueMask, kernel, iterations=1)

    # Show the binary mask for debugging
    cv2.imshow("Blue Mask", blueMask)

    # Find contours in the binary mask
    contours, _ = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Print the number of contours found
    print(f"Number of contours found: {len(contours)}")

    # If at least one contour is found
    if len(contours) > 0:
        # Sort the contours by area and get the largest one
        cnt = sorted(contours, key=cv2.contourArea, reverse=True)[0]

        # Check the area of the largest contour
        contour_area = cv2.contourArea(cnt)
        print(f"Largest contour area: {contour_area}")

        # Only proceed if the contour area is significant
        if contour_area > 500:
            # Get the radius of the minimum enclosing circle around the largest contour
            (x, y), radius = cv2.minEnclosingCircle(cnt)

            # Draw the circle around the largest contour
            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)

            # Calculate the moments of the contour to find the center
            M = cv2.moments(cnt)
            if M["m00"] != 0:  # Ensure that the contour area is not zero to avoid division by zero
                center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
                print(f"Center of contour: {center}")
            else:
                center = (0, 0)
        else:
            print("Contour is too small to be considered.")
    else:
        print("No contours found.")

    # Show the frame with the drawn contour
    cv2.imshow("Tracked Frame", frame)

    # Exit the loop when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the camera and close all OpenCV windows
camera.release()
cv2.destroyAllWindows()

Number of contours found: 0
No contours found.
Number of contours found: 2
Largest contour area: 88.0
Contour is too small to be considered.
Number of contours found: 0
No contours found.
Number of contours found: 0
No contours found.
Number of contours found: 0
No contours found.
Number of contours found: 1
Largest contour area: 63.5
Contour is too small to be considered.
Number of contours found: 1
Largest contour area: 116.5
Contour is too small to be considered.
Number of contours found: 0
No contours found.
Number of contours found: 1
Largest contour area: 175.5
Contour is too small to be considered.
Number of contours found: 2
Largest contour area: 40.0
Contour is too small to be considered.
Number of contours found: 0
No contours found.
Number of contours found: 1
Largest contour area: 90.0
Contour is too small to be considered.
Number of contours found: 1
Largest contour area: 122.5
Contour is too small to be considered.
Number of contours found: 1
Largest contour area: 64.0
Co

KeyboardInterrupt: 

In [23]:
import cv2
import numpy as np
from collections import deque

# Initialize the webcam
camera = cv2.VideoCapture(0)

# Define the color range for blue in HSV space (adjust as needed)
blueLower = np.array([100, 60, 60])
blueUpper = np.array([140, 255, 255])

# Define the kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Initialize a deque to store the points (representing the path)
path_points = deque(maxlen=1024)

# Create a white canvas for the paint application
paintWindow = np.zeros((480, 640, 3), dtype=np.uint8) + 255

# Loop through frames from the webcam
while True:
    # Capture the current frame
    grabbed, frame = camera.read()

    # Break the loop if the frame is not grabbed (e.g., the camera is disconnected)
    if not grabbed:
        break

    # Flip the frame horizontally to avoid mirror effect
    frame = cv2.flip(frame, 1)

    # Convert the frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create a binary mask where the blue colors are white and everything else is black
    blueMask = cv2.inRange(hsv, blueLower, blueUpper)

    # Perform morphological operations to reduce noise in the binary mask
    blueMask = cv2.erode(blueMask, kernel, iterations=2)
    blueMask = cv2.morphologyEx(blueMask, cv2.MORPH_OPEN, kernel)
    blueMask = cv2.dilate(blueMask, kernel, iterations=1)

    # Find contours in the binary mask
    contours, _ = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # If at least one contour is found
    if len(contours) > 0:
        # Sort the contours by area and get the largest one
        cnt = sorted(contours, key=cv2.contourArea, reverse=True)[0]

        # Get the radius of the minimum enclosing circle around the largest contour
        (x, y), radius = cv2.minEnclosingCircle(cnt)

        # Only proceed if the radius is larger than a threshold
        if radius > 10:
            # Calculate the moments of the contour to find the center
            M = cv2.moments(cnt)
            if M["m00"] != 0:  # Ensure that the contour area is not zero to avoid division by zero
                center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

                # Add the center to the deque
                path_points.appendleft(center)

                # Draw the circle around the largest contour
                cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)

    # Draw the path on the paint window by connecting the points in the deque
    for i in range(1, len(path_points)):
        if path_points[i - 1] is None or path_points[i] is None:
            continue
        # Draw a line connecting the points
        cv2.line(paintWindow, path_points[i - 1], path_points[i], (0, 0, 255), 2)

    # Display the frame and the paint window
    cv2.imshow("Paint Application", paintWindow)
    cv2.imshow("Tracked Frame", frame)

    # Exit the loop when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the camera and close all OpenCV windows
camera.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 